In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import fdrcorrection

In [2]:
HLA_table = pd.read_csv("DGN_HLA_df.csv").set_index("patid").apply(lambda x: x > 0).astype("float")
HLA_table

,DQA1*01:01,DQA1*01:02,DQA1*01:03,DQA1*01:04,DQA1*01:05,DQA1*02:01,DQA1*03:01,DQA1*03:02,DQA1*03:03,DQA1*04:01,...,DQB1*04:02,DQB1*05:01,DQB1*05:02,DQB1*05:03,DQB1*05:04,DQB1*06:01,DQB1*06:02,DQB1*06:03,DQB1*06:04,DQB1*06:09
patid,,,,,,,,,,,,,,,,,,,,,
LD0014,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LD0041,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
LD0038,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
LD0084,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
LD0022,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LD1282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LD1271,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
LD1252,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [3]:
family_counts_table = pd.read_csv("DGN_family_counts_table.csv")
family_counts_table = family_counts_table.set_index("patid")
family_counts_table = family_counts_table[[x for x in family_counts_table.columns if x.startswith("TRAV")]]
family_counts_table

,TRAV1-1,TRAV1-2,TRAV10,TRAV12-1,TRAV12-2,TRAV12-3,TRAV13-1,TRAV13-2,TRAV14/DV4,TRAV16,...,TRAV5,TRAV6,TRAV7,TRAV8-1,TRAV8-2,TRAV8-3,TRAV8-4,TRAV8-6,TRAV9-1,TRAV9-2
patid,,,,,,,,,,,,,,,,,,,,,
LD0001,60.0,125.0,94.0,220.0,371.0,202.0,333.0,132.0,187.0,90.0,...,79.0,78.0,0.0,107.0,60.0,169.0,241.0,157.0,0.0,409.0
LD0002,66.0,217.0,113.0,409.0,376.0,243.0,500.0,201.0,187.0,119.0,...,76.0,76.0,0.0,96.0,90.0,213.0,313.0,240.0,1.0,527.0
LD0003,118.0,207.0,82.0,238.0,571.0,269.0,437.0,135.0,263.0,103.0,...,103.0,61.0,0.0,73.0,101.0,153.0,280.0,185.0,0.0,330.0
LD0006,106.0,256.0,110.0,304.0,312.0,246.0,439.0,209.0,169.0,147.0,...,96.0,133.0,0.0,120.0,99.0,194.0,294.0,214.0,1.0,412.0
LD0007,75.0,202.0,114.0,408.0,352.0,276.0,693.0,280.0,364.0,138.0,...,120.0,129.0,0.0,114.0,110.0,244.0,360.0,182.0,0.0,689.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LD1357,89.0,260.0,80.0,191.0,287.0,164.0,281.0,121.0,98.0,69.0,...,70.0,39.0,0.0,71.0,57.0,101.0,194.0,167.0,0.0,295.0
LD1361,66.0,131.0,170.0,329.0,291.0,257.0,650.0,346.0,257.0,164.0,...,123.0,114.0,0.0,79.0,108.0,217.0,307.0,268.0,0.0,581.0
LD1362,59.0,293.0,52.0,265.0,256.0,205.0,361.0,136.0,155.0,102.0,...,111.0,79.0,0.0,63.0,57.0,142.0,242.0,159.0,0.0,238.0


In [4]:
tcr_columns = list(family_counts_table.columns)

In [5]:
CDR_NUM_AA = 22 # 22 positions, first 12 are CDR1, then 10 are CDR2
DGN_NUM_PATIENTS = 895

In [6]:
TRV = "TRBV" # TRAV

positions_AA_count_dfs = []
for i in range(CDR_NUM_AA):
    positions_AA_count_dfs.append(pd.read_csv("amino_acids_correlation/DGN_{}_CDR1_CDR2_position_{}_AA_counts_table.csv".format(TRV, i)).set_index("patid"))

In [7]:
positions_AA_count_dfs[1]

,D,E,F,G,N,Q,R,S,T
patid,,,,,,,,,
LD0001,253.0,502.0,915.0,4322.0,2063.0,498.0,105.0,81.0,89.0
LD0002,480.0,807.0,1035.0,5966.0,3591.0,747.0,224.0,163.0,173.0
LD0003,144.0,644.0,594.0,4553.0,2780.0,307.0,112.0,119.0,256.0
LD0006,739.0,687.0,1416.0,5127.0,3195.0,815.0,194.0,107.0,163.0
LD0007,606.0,891.0,1298.0,7370.0,3882.0,758.0,138.0,138.0,220.0
...,...,...,...,...,...,...,...,...,...
LD1357,331.0,445.0,760.0,4458.0,2111.0,654.0,185.0,55.0,287.0
LD1361,277.0,1075.0,1408.0,7654.0,4325.0,1189.0,183.0,186.0,227.0
LD1362,613.0,731.0,764.0,4410.0,2579.0,511.0,126.0,76.0,216.0


## Using family counts and TRAV sequences, for every patient, get count of each AA at each position of CDR2

In [8]:
def counts_to_usage_table(counts_table, drop_columns={}):
    counts_table = counts_table.drop(axis=1, labels=drop_columns)
    usage_table = counts_table.div(counts_table.sum(axis=1), axis=0)
    return usage_table

In [9]:
family_usage_table = counts_to_usage_table(family_counts_table)

In [10]:
family_usage_table

,TRAV1-1,TRAV1-2,TRAV10,TRAV12-1,TRAV12-2,TRAV12-3,TRAV13-1,TRAV13-2,TRAV14/DV4,TRAV16,...,TRAV5,TRAV6,TRAV7,TRAV8-1,TRAV8-2,TRAV8-3,TRAV8-4,TRAV8-6,TRAV9-1,TRAV9-2
patid,,,,,,,,,,,,,,,,,,,,,
LD0001,0.010435,0.021739,0.016348,0.038261,0.064522,0.035130,0.057913,0.022957,0.032522,0.015652,...,0.013739,0.013565,0.000000,0.018609,0.010435,0.029391,0.041913,0.027304,0.000000,0.071130
LD0002,0.007591,0.024957,0.012996,0.047039,0.043243,0.027947,0.057504,0.023117,0.021507,0.013686,...,0.008741,0.008741,0.000000,0.011041,0.010351,0.024497,0.035998,0.027602,0.000115,0.060610
LD0003,0.016874,0.029601,0.011726,0.034034,0.081653,0.038467,0.062491,0.019305,0.037609,0.014729,...,0.014729,0.008723,0.000000,0.010439,0.014443,0.021879,0.040040,0.026455,0.000000,0.047190
LD0006,0.013472,0.032537,0.013981,0.038638,0.039654,0.031266,0.055796,0.026563,0.021479,0.018683,...,0.012201,0.016904,0.000000,0.015252,0.012583,0.024657,0.037367,0.027199,0.000127,0.052364
LD0007,0.008087,0.021781,0.012292,0.043994,0.037956,0.029761,0.074725,0.030192,0.039250,0.014880,...,0.012939,0.013910,0.000000,0.012292,0.011861,0.026310,0.038818,0.019625,0.000000,0.074294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LD1357,0.017994,0.052568,0.016175,0.038617,0.058027,0.033158,0.056814,0.024464,0.019814,0.013951,...,0.014153,0.007885,0.000000,0.014355,0.011524,0.020421,0.039224,0.033765,0.000000,0.059644
LD1361,0.007784,0.015450,0.020050,0.038802,0.034320,0.030310,0.076660,0.040807,0.030310,0.019342,...,0.014506,0.013445,0.000000,0.009317,0.012737,0.025593,0.036207,0.031608,0.000000,0.068522
LD1362,0.010617,0.052726,0.009358,0.047688,0.046068,0.036890,0.064963,0.024474,0.027893,0.018355,...,0.019975,0.014216,0.000000,0.011337,0.010257,0.025553,0.043549,0.028613,0.000000,0.042829


In [11]:
def do_ols(pat_info_df, usage_table, ind_var, y_cols, cov_cols):
    # ind_var: independent variable for model, e.g. 'DQB1*03:01'
    all_df = pd.merge(pat_info_df, usage_table, right_index=True, left_index=True, how="inner")
#     if binary: # code variable as 0 1
#         ind_var_col_name = "{}_binary".format(ind_var)
#         all_df[ind_var_col_name] = (all_df[ind_var] > 0).astype("float")
#     else: # code variable as 0 1 2
    ind_var_col_name = ind_var
    #all_df[ind_var_col_name] = all_df[ind_var_col_name].astype("float")
    
    #print("doing {} tests. out of {} rows, {} are {} > 0".format(len(y_cols), len(all_df), len(all_df[all_df[ind_var]>0]), ind_var))
    x_columns = [ind_var_col_name] + cov_cols
    res_df = pd.DataFrame()
    for y_column in y_cols:
        X = all_df[x_columns]
        y = all_df[y_column]
        X = sm.add_constant(X) # constant is always added
        mod = sm.OLS(y, X)
        res = mod.fit()
        #print(res.summary())
        tcr_df = pd.DataFrame()
        tcr_df['coef'] = res.params
        tcr_df['se'] = res.bse
        tcr_df['pvalue'] = res.pvalues
        tcr_df['tvalue'] = res.tvalues
        res_df[y_column] = tcr_df.loc[ind_var_col_name]
    res_df = res_df.transpose().reset_index().rename(columns={"index":"family"})
    # FDR correction
    res_df['pvalue_fdr'] = fdrcorrection(res_df['pvalue'])[1]
    return res_df

In [12]:
def do_stepwise_ols(HLA_table, counts_table, ind_var, want_families={}, fdr=True):
    
    def is_continue(df):
        if len(want_families) > 0: # need results for all families in want_families
            print(set(df['family']).intersection(want_families), want_families)
            if set(df['family']).intersection(want_families) == want_families:
                return False
        return True
    
    df = pd.DataFrame(columns=["family", "coef", "se", "pvalue", "tvalue"])
    drop_columns = []

    while is_continue(df):
        usage_table = counts_to_usage_table(counts_table, drop_columns=drop_columns)
        if len(usage_table.columns) == 0: # dropped all the columns already, nothing left
            break
        res_df = do_ols(HLA_table, usage_table, ind_var, usage_table.columns, [])
        top_row = res_df.iloc[0]
        if fdr:
            pvalue = top_row['pvalue_fdr']
        else:
            pvalue = top_row['pvalue']
        if len(want_families) == 0:
            if pvalue > 0.1: # get all rows until the top row is no longer significant
                break
        print("{}, coef: {}, pvalue: {}".format(top_row['family'], top_row['coef'], pvalue))
        drop_columns.append(top_row['family'])
        df = df.append(top_row)
    df['coef'] = df['coef']*100 # want the coef to be in percentages
    return df

In [ ]:
# #usage_df['usage_ratio'] = usage_df['usage_ratio']*100 # only do once!
# cov_df = pd.read_csv("DGN_covariates_df.csv").set_index("patid")
# cov_df["lane"] = cov_df["fcid"] + "_" + cov_df["lane"].astype("str") # covariates
# cov_df = pd.get_dummies(cov_df, columns=['fcid', 'lane'])
# pat_info_df = pd.merge(cov_df, HLA_table, right_index=True, left_index=True, how="inner")

In [ ]:
# step_res_df_301_all = do_stepwise_ols(HLA_table, family_counts_table, ind_var="DQB1*03:01", fdr=True)

# step_res_df_301_all

In [ ]:
# tcr_columns = list(usage_table.columns)
# do_ols(HLA_table, usage_table, ind_var="DQB1*03:01", y_cols=tcr_columns, cov_cols=[])

## Families OLS

In [21]:
for family in ["TRAV", "TRBV", "TRAJ", "TRBJ"]:
    family_counts_table = pd.read_csv("DGN_family_counts_table.csv")
    family_counts_table = family_counts_table.set_index("patid")
    family_counts_table = family_counts_table[[x for x in family_counts_table.columns if x.startswith(family)]]
    family_usage_table = counts_to_usage_table(family_counts_table)
    family_res_df = do_ols(HLA_table, family_usage_table, "DQB1*03:01", family_usage_table.columns, [])
    
    # separately compute DQB0301_pos_mean_usage and DQB0301_neg_mean_usage and join in to results
    all_df = pd.merge(HLA_table, family_usage_table, right_index=True, left_index=True, how="inner")
    num_DQB0301_pos = len(all_df[all_df["DQB1*03:01"]==1.0])
    num_DQB0301_neg = len(all_df[all_df["DQB1*03:01"]!=1.0])
    DQB0301_pos_mean_usage = all_df[all_df["DQB1*03:01"]==1.0][[x for x in family_counts_table.columns if x.startswith(family)]].mean().to_frame(name=f"DQB0301_pos_mean_usage").reset_index().rename(columns={'index': 'family'})
    DQB0301_neg_mean_usage = all_df[all_df["DQB1*03:01"]!=1.0][[x for x in family_counts_table.columns if x.startswith(family)]].mean().to_frame(name=f"DQB0301_neg_mean_usage").reset_index().rename(columns={'index': 'family'})
    mean_usage_df = DQB0301_pos_mean_usage.merge(DQB0301_neg_mean_usage, how="inner", on="family")
    mean_usage_df["DQB0301_pos_to_neg_mean_usage_ratio"] = mean_usage_df["DQB0301_pos_mean_usage"] / mean_usage_df["DQB0301_neg_mean_usage"]
    merged_df = mean_usage_df.merge(family_res_df, how="inner", on="family")
    merged_df = merged_df.sort_values(by=['pvalue_fdr'])
    merged_df = merged_df.rename(columns={"DQB0301_pos_mean_usage": f"DQB0301_pos_mean_usage (n={num_DQB0301_pos})", "DQB0301_neg_mean_usage": f"DQB0301_neg_mean_usage (n={num_DQB0301_neg})"})
    merged_df.to_csv(f"FINAL_DGN_{family}_FAMILY_OLS.csv", sep="\t", index=False)

## Families Stepwise OLS

In [ ]:
step_family_res_df = do_stepwise_ols(HLA_table, family_counts_table, "DQB1*03:01")
step_family_res_df.to_csv("FINAL_DGN_FAMILY_STEPWISE_OLS.csv", sep="\t")

In [ ]:
cond_A = HLA_table['DQB1*03:01'] > 0
cond_B = HLA_table['DQB1*03:02'] > 0
filt_HLA_table = HLA_table[(cond_A & ~cond_B) | (cond_B & ~cond_A)]
step_res_df_301_302 = do_stepwise_ols(filt_HLA_table, family_counts_table, "DQB1*03:01", fdr=True, covariates=False, binary=True)

In [ ]:
step_res_df_301_302

In [ ]:
cond_A = HLA_table['DQB1*03:01'] > 0
cond_B = HLA_table['DQB1*06:01'] > 0
filt_HLA_table = HLA_table[(cond_A & ~cond_B) | (cond_B & ~cond_A)]
step_res_df_301_302 = do_stepwise_ols(filt_HLA_table, family_counts_table, "DQB1*03:01", fdr=True, covariates=False, binary=True)

## Amino Acids OLS

In [13]:
ind_var = "DQB1*03:01"
#print("indicator variable is {}".format(ind_var))

res_df_lst = []

## CDR1
TRAV_active_positions = [0, 1, 2, 3, 9, 10, 11]
TRBV_active_positions = [0, 1, 2, 9, 10, 11]

if TRV == "TRAV":
    TRV_active_positions = TRAV_active_positions
else:
    TRV_active_positions = TRBV_active_positions

#print(TRV, "CDR1")
#print("everyone with {} is also {}".format(ind_var, filt_col))
for i in TRV_active_positions:
    #print("--- position {} ---".format(i+27))
    position_AA_usage_table = counts_to_usage_table(positions_AA_count_dfs[i])
    res_df = do_ols(HLA_table, position_AA_usage_table, ind_var, position_AA_usage_table.columns, [])
    res_df["position"] = i+27
    res_df_lst.append(res_df)

#print("\n")
## CDR2
TRAV_active_positions = [0, 1, 2, 3, 6, 7, 8, 9]
TRBV_active_positions = [0, 1, 2, 3, 7, 8, 9]

if TRV == "TRAV":
    TRV_active_positions = TRAV_active_positions
else:
    TRV_active_positions = TRBV_active_positions

#print(TRV, "CDR2")
#print("everyone with {} is also {}".format(ind_var, filt_col))
for i in TRV_active_positions:
    #print("--- position {} ---".format(i+56))
    # +12 to offset the CDR1 AA's in front
    position_AA_usage_table = counts_to_usage_table(positions_AA_count_dfs[i+12])
    res_df = do_ols(HLA_table, position_AA_usage_table, ind_var, position_AA_usage_table.columns, [])
    res_df["position"] = i+56
    res_df_lst.append(res_df)
    
chi_square_AA_res_df = pd.concat(res_df_lst)
chi_square_AA_res_df = chi_square_AA_res_df[["position", "family", "coef", "pvalue"]]
chi_square_AA_res_df['pvalue_fdr'] = fdrcorrection(chi_square_AA_res_df['pvalue'])[1]

In [14]:
chi_square_AA_res_df[chi_square_AA_res_df["pvalue_fdr"] < 0.05].sort_values(by=['position', 'pvalue_fdr']).to_csv(f"FINAL_DGN_{TRV}_AMINOACID_OLS.csv", sep="\t", index=None)

## Amino Acids Stepwise OLS

In [ ]:
#print("everyone is either DQA1*03:03 or DQA1*05:05")
ind_var = "DQB1*03:01" #"DQA1*05:05" 
print("indicator variable is {}".format(ind_var))
# for filt_col in ['DQA1*05:05', 'DQA1*03:03']:
print("\n")
#HLA_table_filt = HLA_table[(HLA_table['DQA1*03:03'] == 1) | (HLA_table['DQA1*05:05'] == 1)]
# HLA_table_filt = HLA_table[~((HLA_table['DQB1*03:01'] == 1) & (HLA_table[filt_col] != 1))]
# HLA_table_filt = HLA_table[HLA_table[filt_col] == 1.0] # HLA_table[HLA_table['DQA1*03:03'] == 1.0]
HLA_table_filt = HLA_table

## CDR1
TRAV_active_positions = [0, 1, 2, 3, 9, 10, 11]
TRBV_active_positions = [0, 1, 2, 9, 10, 11]

if TRV == "TRAV":
    TRV_active_positions = TRAV_active_positions
else:
    TRV_active_positions = TRBV_active_positions

print(TRV, "CDR1")
#print("everyone with {} is also {}".format(ind_var, filt_col))
for i in TRV_active_positions:
    print("--- position {} ---".format(i+27))
    do_stepwise_ols(HLA_table_filt, positions_AA_count_dfs[i], ind_var=ind_var, fdr=True)

print("\n")
## CDR2
TRAV_active_positions = [0, 1, 2, 3, 6, 7, 8, 9]
TRBV_active_positions = [0, 1, 2, 3, 7, 8, 9]

if TRV == "TRAV":
    TRV_active_positions = TRAV_active_positions
else:
    TRV_active_positions = TRBV_active_positions

print(TRV, "CDR2")
#print("everyone with {} is also {}".format(ind_var, filt_col))
for i in TRV_active_positions:
    print("--- position {} ---".format(i+56))
    # +12 to offset the CDR1 AA's in front
    do_stepwise_ols(HLA_table_filt, positions_AA_count_dfs[i+12], ind_var=ind_var, fdr=True)

In [ ]:
positions_AA_count_dfs[0]